# Retrieve MUR SST data at M8

(python2 only - may be urllib call?)

In [1]:
%matplotlib inline

import datetime

import pandas as pd
import xarray as xa
from netCDF4 import Dataset

import urllib

import cmocean

/Users/bell/anaconda2/lib/python2.7/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
### using code from https://ioos.github.io/notebooks_demos/notebooks/2017-03-21-ERDDAP_IOOS_Sensor_Map/
import requests
try:
    from urllib.parse import urlencode
except ImportError:
    from urllib import urlencode

#MUR JPL data
erddap_path_mur="http://coastwatch.pfeg.noaa.gov/erddap/griddap/jplMURSST41"
e1rdf = xa.open_dataset(erddap_path_mur) 


In [3]:
sst_mur = e1rdf['analysed_sst'].sel(time=slice('2017-12-01','2018-04-22'),
                                        latitude=slice(62.189,62.191),
                                        longitude=slice(-174.691,-174.689))

In [4]:
sst_mur = e1rdf['analysed_sst']

In [ ]:
sst_mur.to_series().to_csv()

### Using erddapy (and python3)

Get some dataset information first

In [2]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np

server_url='http://coastwatch.pfeg.noaa.gov/erddap'
e = ERDDAP(server=server_url)

In [4]:
e.constraints = None
e.protocol = 'griddap'

opendap_url = e.get_download_url(
    dataset_id='jplMURSST41',
)

print(opendap_url)

http://coastwatch.pfeg.noaa.gov/erddap/griddap/jplMURSST41


In [5]:
from netCDF4 import Dataset

with Dataset(opendap_url) as nc:
    print(nc.summary)

This is a merged, multi-sensor L4 Foundation Sea Surface Temperature (SST) analysis product from Jet Propulsion Laboratory (JPL). This daily, global, Multi-scale, Ultra-high Resolution (MUR) Sea Surface Temperature (SST) 1-km data set, Version 4.1, is produced at JPL under the NASA MEaSUREs program. For details, see https://podaac.jpl.nasa.gov/dataset/MUR-JPL-L4-GLOB-v4.1 . This dataset is part of the Group for High-Resolution Sea Surface Temperature (GHRSST) project. The data for the most recent 4 days is usually revised everyday.  The data for other days is sometimes revised.


In [6]:
kw = {
    'standard_name': 'sea_water_temperature',
    'min_lon': -174.691,
    'max_lon': -174.689,
    'min_lat': 62.189,
    'max_lat': 62.191,
    'min_time': '2015-01-10T00:00:00Z',
    'max_time': '2019-01-10T00:00:00Z',
    'cdm_data_type': 'trajectory'
}

In [8]:
common_variables = set(['latitude', 'longitude','time','analysed_sst'])

constraints = {
    'longitude>=': kw['min_lon'],
    'longitude<=': kw['max_lon'],
    'latitude>=': kw['min_lat'],
    'latitude<=': kw['max_lat'],
    'time>=': kw['min_time'],
    'time<=': kw['max_time'],
}

In [9]:
e.dataset_id = 'jplMURSST41'
e.response = 'nc'
e.variables = common_variables
e.constraints = constraints

download_url = e.get_download_url()

HTTPError: 500 Server Error:  for url: http://coastwatch.pfeg.noaa.gov/erddap/griddap/jplMURSST41.nc?latitude,analysed_sst,longitude,time&longitude%3E=-174.691&longitude%3C=-174.689&latitude%3E=62.189&latitude%3C=62.191&time%3E=1420848000.0&time%3C=1547078400.0

*** looks like griddap is the challenge here ***